# Train simple CNN classifier
Here we train a CNN to predict cancer grade because on tiled histological images. See the following notebook for the generation of these images:  
https://www.kaggle.com/lvulliard/crop-images

In [ ]:
import os
import shutil

# There are two ways to load the data from the PANDA dataset:
# Option 1: Load images using openslide
import openslide
# Option 2: Load images using skimage (requires that tifffile is installed)
import skimage.io

# General packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import PIL
from IPython.display import Image, display
from collections import Counter

import cv2
import skimage.io
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
import tensorflow as tf
tf.test.is_gpu_available()

## Install Efficient-net
Resources collected here: https://www.kaggle.com/prateekagnihotri/efficientnet-keras-train-qwk-loss-augmentation


In [ ]:
!pip install -qq /kaggle/input/sequencedata/efficientnet-1.1.0-py3-none-any.whl

## Set directories and parameters

In [ ]:
# Location of the training images
dataDir = '/kaggle/input/crop-images/cropped_train_images/'

# Location of training labels
trainLabels = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/train.csv').set_index('image_id')
testDF = pd.read_csv('/kaggle/input/prostate-cancer-grade-assessment/test.csv').set_index('image_id')

# Output cropped images
cropDir = '/kaggle/working/cropped_train_images/'

inputShape = (224, 224, 3)
epochs = 100

In [ ]:
# How many train objects should be included in one batch (higher = faster but less accurate)
# Take care that the batch size is smaller than the amount of total images analyzed
batchSize = 256
INIT_LR = 1e-4

## Data generators

In [ ]:
trainDatagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, validation_split = 0.1,
                                  zoom_range=0.2, horizontal_flip=True, fill_mode="nearest")

In [ ]:
trainDF = pd.DataFrame(list(zip(trainLabels.index + ".png", trainLabels.isup_grade.astype(str))), 
               columns =['x_col', 'y_col']) 

# Uncomment the following if the assumption needs to be re-checked
# for x in trainDF.x_col:
#     assert x in os.listdir(dataDir)

In [ ]:
trainGenerator = trainDatagen.flow_from_dataframe(
    trainDF, x_col="x_col", y_col="y_col",
    directory=dataDir,  # this is the target directory
    batch_size=batchSize,
    class_mode = "sparse",
    subset="training",
    target_size=(inputShape[0], inputShape[1]),
    color_mode='rgb')

In [ ]:
valGenerator = trainDatagen.flow_from_dataframe(
    trainDF, x_col="x_col", y_col="y_col",
    directory=dataDir,  # this is the target directory
    batch_size=batchSize,
    class_mode = "sparse",
    subset="validation",
    target_size=(inputShape[0], inputShape[1]),
    color_mode='rgb')

## EfficientNet model
See https://www.kaggle.com/hassanamin/transfer-learning-vgg16-examples-using-tensorflow#Specify-the-Model

In [ ]:
import efficientnet.keras as efn 
from keras.models import Sequential
from keras.layers import Dense, Flatten, GlobalAveragePooling2D

In [ ]:
numClasses = len(set(trainLabels.isup_grade))
weightFile = "/kaggle/input/efficientnet-weights-for-keras/noisy-student/notop/efficientnet-b1_noisy-student_notop.h5"

myModel = Sequential()
myModel.add(efn.EfficientNetB1(include_top=False, pooling='avg', weights=weightFile))
myModel.add(Dense(numClasses, activation='softmax'))

# Say not to train first layer (pre-trainerd model)
myModel.layers[0].trainable = False

In [ ]:
myModel.summary()

In [ ]:
# Optimaztion function
opt = Adam(lr=INIT_LR, decay=INIT_LR / epochs)

myModel.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["binary_accuracy"])

In [ ]:
H = myModel.fit_generator(trainGenerator,
                        steps_per_epoch=len(trainLabels.index) // batchSize,
                        epochs=epochs, 
                        validation_data=valGenerator,
                        validation_steps=1,
                        verbose=1)

In [ ]:
# Save model
myModel.save('/kaggle/working/B1_'+str(epochs)+'.model')